In [7]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
from nibabel.processing import resample_from_to  # part of nibabel >=3.2

# ---------- CONFIGURE PATHS ----------
ct_path = '/home/abigail/mphys/CT_WHOLE_CNS.nii'
seg_path = '/home/abigail/mphys/teeth_upper.nii.gz'

# ---------- LOAD DATA ----------
ct_img = nib.load(ct_path)
seg_img = nib.load(seg_path)

# ---------- RESAMPLE SEGMENTATION TO MATCH CT ----------
seg_resampled = resample_from_to(seg_img, ct_img, order=0)  # nearest-neighbor for labels
ct_data = ct_img.get_fdata()
seg_data = seg_resampled.get_fdata()

# ---------- NORMALIZE CT ----------
ct_min, ct_max = np.percentile(ct_data, (1, 99))
ct_norm = np.clip((ct_data - ct_min) / (ct_max - ct_min), 0, 1)

n_slices = ct_data.shape[2]

# ---------- VISUALIZATION ----------
def show_slice(slice_idx):
    plt.figure(figsize=(6,6))
    plt.imshow(ct_norm[:, :, slice_idx].T, cmap="gray", origin="lower")
    
    seg_mask = seg_data[:, :, slice_idx]
    if np.any(seg_mask):
        plt.imshow(np.ma.masked_where(seg_mask == 0, seg_mask).T,
                   cmap="Reds", alpha=0.4, origin="lower")
    
    plt.title(f"Slice {slice_idx}/{n_slices-1}")
    plt.axis("off")
    plt.show()

interact(show_slice, slice_idx=IntSlider(min=0, max=n_slices-1, step=1, value=n_slices//2))


interactive(children=(IntSlider(value=261, description='slice_idx', max=521), Output()), _dom_classes=('widget…

<function __main__.show_slice(slice_idx)>

In [ ]:
import nibabel as nib
import numpy as np
from nibabel.processing import resample_from_to
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

# ---------- Load ----------
ct_path = "/home/abigail/mphys/CT_WHOLE_CNS.nii"
seg_path = "/home/abigail/mphys/teeth_upper.nii.gz"

ct_img = nib.load(ct_path)
seg_img = nib.load(seg_path)

# ---------- Resample segmentation to CT space ----------
# This will automatically handle the affine offsets
seg_resampled = resample_from_to(seg_img, ct_img, order=0)  # nearest-neighbor resampling
seg_data = seg_resampled.get_fdata()
ct_data = ct_img.get_fdata()

# ---------- Normalize CT ----------
ct_min, ct_max = np.percentile(ct_data, (1, 99))
ct_norm = np.clip((ct_data - ct_min) / (ct_max - ct_min), 0, 1)

# ---------- Visualization ----------
n_slices = ct_data.shape[2]

def show_slice(slice_idx):
    plt.figure(figsize=(6,6))
    plt.imshow(ct_norm[:, :, slice_idx].T, cmap="gray", origin="lower")
    
    mask = seg_data[:, :, slice_idx]
    if np.any(mask):
        plt.imshow(np.ma.masked_where(mask == 0, mask).T,
                   cmap="autumn", alpha=0.5, origin="lower")
    
    plt.title(f"Slice {slice_idx}/{n_slices-1}")
    plt.axis("off")
    plt.show()

interact(show_slice, slice_idx=IntSlider(min=0, max=n_slices-1, step=1, value=330))


interactive(children=(IntSlider(value=330, description='slice_idx', max=521), Output()), _dom_classes=('widget…

<function __main__.show_slice(slice_idx)>

In [ ]:
import nibabel as nib
import numpy as np
from nibabel.processing import resample_from_to
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

# ---------- Load ----------
ct_path = "/home/abigail/mphys/CT_WHOLE_CNS.nii"
seg_path = "/home/abigail/mphys/teeth_upper.nii.gz"

ct_img = nib.load(ct_path)
seg_img = nib.load(seg_path)

# ---------- Resample segmentation into CT grid ----------
seg_resampled = resample_from_to(seg_img, ct_img, order=0)
seg_data = seg_resampled.get_fdata()
ct_data = ct_img.get_fdata()

# ---------- Manual Y shift (posterior) ----------
# Teeth too far forward → shift back (posterior)
shift_voxels_y = 25   # try 30–40 if you want to fine-tune
seg_shifted = np.roll(seg_data, shift_voxels_y, axis=1)

# Zero-out wrapped area to avoid ghosting
seg_shifted[:, :shift_voxels_y, :] = 0

# ---------- Normalize CT ----------
ct_min, ct_max = np.percentile(ct_data, (1, 99))
ct_norm = np.clip((ct_data - ct_min) / (ct_max - ct_min), 0, 1)

# ---------- Interactive viewer ----------
n_slices = ct_data.shape[2]

def show_slice(slice_idx):
    plt.figure(figsize=(6,6))
    plt.imshow(ct_norm[:, :, slice_idx].T, cmap="gray", origin="lower")
    mask = seg_shifted[:, :, slice_idx]
    if np.any(mask):
        plt.imshow(np.ma.masked_where(mask == 0, mask).T,
                   cmap="autumn", alpha=0.5, origin="lower")
    plt.title(f"Slice {slice_idx}/{n_slices-1}")
    plt.axis("off")
    plt.show()

interact(show_slice, slice_idx=IntSlider(min=0, max=n_slices-1, step=1, value=330))


interactive(children=(IntSlider(value=330, description='slice_idx', max=521), Output()), _dom_classes=('widget…

<function __main__.show_slice(slice_idx)>

In [ ]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider
from nibabel.processing import resample_from_to

# ---------- LOAD ----------
ct_path = "/home/abigail/mphys/CT_WHOLE_CNS.nii"
seg_path = "/home/abigail/mphys/teeth_upper.nii.gz"

ct_img = nib.load(ct_path)
seg_img = nib.load(seg_path)

# Resample to CT grid
seg_resampled = resample_from_to(seg_img, ct_img, order=0)
ct_data = ct_img.get_fdata()
seg_data = seg_resampled.get_fdata()

# ---------- NORMALIZE CT ----------
ct_min, ct_max = np.percentile(ct_data, (1, 99))
ct_norm = np.clip((ct_data - ct_min) / (ct_max - ct_min), 0, 1)

n_slices = ct_data.shape[2]

# ---------- INTERACTIVE VIEWER ----------
def show_slice(slice_idx, y_shift=0.0, z_shift=0.0):
    # Convert mm shift to voxel shift
    y_voxel = int(round(y_shift / abs(ct_img.header.get_zooms()[1])))
    z_voxel = int(round(z_shift / abs(ct_img.header.get_zooms()[2])))

    seg_shifted = np.roll(seg_data, y_voxel, axis=1)
    seg_shifted = np.roll(seg_shifted, z_voxel, axis=2)

    # Zero out wrapped edges
    if y_voxel > 0:
        seg_shifted[:, :y_voxel, :] = 0
    elif y_voxel < 0:
        seg_shifted[:, y_voxel:, :] = 0
    if z_voxel > 0:
        seg_shifted[:, :, :z_voxel] = 0
    elif z_voxel < 0:
        seg_shifted[:, :, z_voxel:] = 0

    plt.figure(figsize=(6,6))
    plt.imshow(ct_norm[:, :, slice_idx].T, cmap="gray", origin="lower")
    mask = seg_shifted[:, :, slice_idx]
    if np.any(mask):
        plt.imshow(np.ma.masked_where(mask == 0, mask).T,
                   cmap="autumn", alpha=0.5, origin="lower")
    plt.title(f"Slice {slice_idx} | Y shift: {y_shift:.1f} mm | Z shift: {z_shift:.1f} mm")
    plt.axis("off")
    plt.show()

# ---------- LAUNCH ----------
interact(
    show_slice,
    slice_idx=IntSlider(min=0, max=n_slices-1, step=1, value=n_slices//2),
    y_shift=FloatSlider(min=-50, max=50, step=1, value=0, description='Y shift (mm)'),
    z_shift=FloatSlider(min=-100, max=100, step=2, value=0, description='Z shift (mm)')
)


interactive(children=(IntSlider(value=261, description='slice_idx', max=521), FloatSlider(value=0.0, descripti…

<function __main__.show_slice(slice_idx, y_shift=0.0, z_shift=0.0)>

In [3]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider
from nibabel.processing import resample_from_to

# ---------- CONFIGURE YOUR FILE PATHS ----------
ct_path = "/home/abigail/mphys/CT_WHOLE_CNS.nii"

# Add as many segmentations as you want here
seg_paths = {
    "teeth_upper": "/home/abigail/mphys/teeth_upper.nii.gz",
    "teeth_lower": "/home/abigail/mphys/teeth_lower.nii.gz",
    "mandible": "/home/abigail/mphys/mandible.nii.gz",
    "skull": "/home/abigail/mphys/skull.nii.gz",
    "head": "/home/abigail/mphys/head.nii.gz",
    "sinus_frontal": "/home/abigail/mphys/sinus_frontal.nii.gz",
    "sinus_maxillary": "/home/abigail/mphys/sinus_maxillary.nii.gz"
}

# Pick visually distinct colormaps
colors = {
    "teeth_upper": "autumn",
    "teeth_lower": "Oranges",
    "mandible": "Blues",
    "skull": "Greens",
    "head": "Purples",
    "sinus_frontal": "flag",
    "sinus_maxillary": "hsv"
}

# ---------- LOAD AND RESAMPLE SEGMENTATIONS ----------
ct_img = nib.load(ct_path)
ct_data = ct_img.get_fdata()

# Resample all segmentations to CT space
seg_data_dict = {}
for name, path in seg_paths.items():
    try:
        seg_img = nib.load(path)
        seg_resampled = resample_from_to(seg_img, ct_img, order=0)
        seg_data_dict[name] = seg_resampled.get_fdata()
        print(f"Loaded and aligned: {name}")
    except Exception as e:
        print(f"❌ Failed to load {name}: {e}")

# ---------- NORMALIZE CT ----------
ct_min, ct_max = np.percentile(ct_data, (1, 99))
ct_norm = np.clip((ct_data - ct_min) / (ct_max - ct_min), 0, 1)

n_slices = ct_data.shape[2]

# ---------- VIEWER FUNCTION ----------
def show_slice(slice_idx, y_shift=0.0, z_shift=0.0):
    # Convert mm shift → voxel shift
    zooms = ct_img.header.get_zooms()
    y_vox = int(round(y_shift / abs(zooms[1])))
    z_vox = int(round(z_shift / abs(zooms[2])))

    plt.figure(figsize=(6, 6))
    plt.imshow(ct_norm[:, :, slice_idx].T, cmap="gray", origin="lower")

    for name, seg_data in seg_data_dict.items():
        # Shift segmentation
        shifted = np.roll(seg_data, y_vox, axis=1)
        shifted = np.roll(shifted, z_vox, axis=2)

        # Zero-out wrapped edges to avoid artifacts
        if y_vox > 0:
            shifted[:, :y_vox, :] = 0
        elif y_vox < 0:
            shifted[:, y_vox:, :] = 0
        if z_vox > 0:
            shifted[:, :, :z_vox] = 0
        elif z_vox < 0:
            shifted[:, :, z_vox:] = 0

        mask = shifted[:, :, slice_idx]
        if np.any(mask):
            plt.imshow(np.ma.masked_where(mask == 0, mask).T,
                       cmap=colors.get(name, "autumn"), alpha=0.4, origin="lower", interpolation="none")

    plt.title(f"Slice {slice_idx} | Y shift: {y_shift:.1f} mm | Z shift: {z_shift:.1f} mm")
    plt.axis("off")
    plt.show()


# ---------- INTERACTIVE CONTROLS ----------
interact(
    show_slice,
    slice_idx=IntSlider(min=0, max=n_slices - 1, step=1, value=n_slices // 2, description="Slice"),
    y_shift=FloatSlider(min=-50, max=50, step=1, value=0, description="Y shift (mm)"),
    z_shift=FloatSlider(min=-100, max=100, step=2, value=0, description="Z shift (mm)")
)


Loaded and aligned: teeth_upper
Loaded and aligned: teeth_lower
Loaded and aligned: mandible
Loaded and aligned: skull
Loaded and aligned: head
Loaded and aligned: sinus_frontal
Loaded and aligned: sinus_maxillary


interactive(children=(IntSlider(value=261, description='Slice', max=521), FloatSlider(value=0.0, description='…

<function __main__.show_slice(slice_idx, y_shift=0.0, z_shift=0.0)>

In [24]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider
from nibabel.processing import resample_from_to

# ---------- CONFIGURE YOUR FILE PATHS ----------
ct_path = "/home/abigail/mphys/CT_WHOLE_CNS.nii"

# Add as many segmentations as you want here
seg_paths = {
    "teeth_upper": "/home/abigail/mphys/teeth_upper.nii.gz",
    "teeth_lower": "/home/abigail/mphys/teeth_lower.nii.gz",
    "mandible": "/home/abigail/mphys/mandible.nii.gz",
    "skull": "/home/abigail/mphys/skull.nii.gz",
    "head": "/home/abigail/mphys/head.nii.gz",
    "sinus_frontal": "/home/abigail/mphys/sinus_frontal.nii.gz",
    "sinus_maxillary": "/home/abigail/mphys/sinus_maxillary.nii.gz"
}

# Pick visually distinct colormaps
colors = {
    "teeth_upper": "autumn",
    "teeth_lower": "Oranges",
    "mandible": "Blues",
    "skull": "Greens",
    "head": "Purples",
    "sinus_frontal": "flag",
    "sinus_maxillary": "hsv"
}

# ---------- LOAD AND RESAMPLE SEGMENTATIONS ----------
ct_img = nib.load(ct_path)
ct_data = ct_img.get_fdata()

# Resample all segmentations to CT space
seg_data_dict = {}
for name, path in seg_paths.items():
    try:
        seg_img = nib.load(path)
        seg_resampled = resample_from_to(seg_img, ct_img, order=0)
        seg_data_dict[name] = seg_resampled.get_fdata()
        print(f"Loaded and aligned: {name}")
    except Exception as e:
        print(f"❌ Failed to load {name}: {e}")

# ---------- NORMALIZE CT ----------
ct_min, ct_max = np.percentile(ct_data, (1, 99))
ct_norm = np.clip((ct_data - ct_min) / (ct_max - ct_min), 0, 1)

n_slices = ct_data.shape[2]
y_vox = int(round(best_y_shift / abs(ct_img.header.get_zooms()[1])))
z_vox = int(round(best_z_shift / abs(ct_img.header.get_zooms()[2])))

seg_shifted_dict = {}
for name, seg_data in seg_data_dict.items():
    shifted = np.roll(seg_data, y_vox, axis=1)
    shifted = np.roll(shifted, z_vox, axis=2)
    # zero out wrapped edges
    if y_vox > 0: shifted[:, :y_vox, :] = 0
    if y_vox < 0: shifted[:, y_vox:, :] = 0
    if z_vox > 0: shifted[:, :, :z_vox] = 0
    if z_vox < 0: shifted[:, :, z_vox:] = 0
    seg_shifted_dict[name] = shifted

def show_slice(slice_idx):
    plt.figure(figsize=(6,6))
    plt.imshow(ct_norm[:, :, slice_idx].T, cmap="gray", origin="lower")
    for name, mask in seg_shifted_dict.items():
        if np.any(mask[:, :, slice_idx]):
            plt.imshow(np.ma.masked_where(mask[:, :, slice_idx] == 0, mask[:, :, slice_idx]).T,
                       cmap=colors.get(name, "autumn"), alpha=0.4, origin="lower")
    plt.axis("off")
    plt.show()

interact(show_slice, slice_idx=IntSlider(min=0, max=n_slices-1, step=1, value=n_slices//2))


Loaded and aligned: teeth_upper
Loaded and aligned: teeth_lower
Loaded and aligned: mandible
Loaded and aligned: skull
Loaded and aligned: head
Loaded and aligned: sinus_frontal
Loaded and aligned: sinus_maxillary


NameError: name 'best_y_shift' is not defined